In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [4]:
def rng(m=2**32, a=1103515245, c=12345):
    rng.current = (a * rng.current + c) % m
    return rng.current / m

# setting the seed
rng.current = 1
samples = [rng() for _ in range(1000)]
print(samples)

[0.25693503906950355, 0.5878706516232342, 0.15432575810700655, 0.767266943352297, 0.9738139626570046, 0.5858681506942958, 0.8511155843734741, 0.6132153405342251, 0.7473867232911289, 0.06236015981994569, 0.04194940160959959, 0.1948235605377704, 0.1386128985323012, 0.6840353596489877, 0.49171859584748745, 0.7676989699248224, 0.8828409514389932, 0.8232365751173347, 0.8835694054141641, 0.8901184604037553, 0.9114757110364735, 0.5759661465417594, 0.3127383701503277, 0.15734241367317736, 0.1734505402855575, 0.45860223821364343, 0.2598799681290984, 0.7005771037656814, 0.3033791915513575, 0.8927010844927281, 0.9657614398747683, 0.9349605368915945, 0.9332623495720327, 0.337260173400864, 0.8791998000815511, 0.7909467287827283, 0.1946238619275391, 0.6778173677157611, 0.6001160368323326, 0.41346341953612864, 0.7079516570083797, 0.23176096356473863, 0.4895814983174205, 0.06321822595782578, 0.10631834948435426, 0.4792256865184754, 0.8687314670532942, 0.7045282314065844, 0.8900565332733095, 0.37894962

### Problem 1
Implement serial test and apply to a `samples`

In [12]:
def serial_test(samples):
    length = 2
    tuples = list(zip(*[samples[i::length] for i in range(length)]))
    x1 = samples[0::length] #1-ая коорда каждой пары
    y1 = samples[1::length] #2-ая

    j = 20 # разбиваем на 20*20=400 бинов гистограмму
    hist, _, _ = np.histogram2d(x1, y1, bins=np.linspace(0, 1, j + 1))

    # подготовка данных для теста хи-квадрат
    f_observed = hist.flatten()
    f_expected = np.full(j * j, len(tuples) / (j * j))

    #cам тест
    print(stats.chisquare(f_observed, f_expected))

serial_test(samples)

Power_divergenceResult(statistic=np.float64(391.2), pvalue=np.float64(0.6003664860752091))


### Problem 2*
Implement permutation test and apply to a `samples`

### Problem 3

Provide a statistical test to show that `rng_bad` has problems but `rng_good` does not.

### Problem 4
Simulate a sample of 1000 from a discrete distribution on a set of digits $0, 1, 2, \ldots, 9$ with weights $0.12, 0.3, 0.167, 0.24, 0.31, 0.54, 0.111, 0.02, 0.001, 0.2$. Construct a histogram from the sample. Optimise the algorithm by ordering the weights.

### Problem 5
Simulate a sample of size 1000 from the distribution $\operatorname{Exp}(\lambda)$ using the Inverse transform method. Construct a sample histogram and an accurate plot of the distribution density.

- **Exponential distribution $\operatorname{Exp}(\lambda)$**:
$$p(x) = \lambda e^{-\lambda x}\cdot\mathbb{I}_{x\geqslant0}$$

### Problem 6
Justify (orally) that the accept-reject method really samples from the required distribution. Simulate $1000$ points from the distribution with density $e^xcos^2x \cdot \mathbb{I}_{[-\frac\pi2, \frac\pi2]}$. Plot the sampled histogram on the sample and compare it with the plot of the exact density function.

насчет orally: ну так если мы задаем выборку с равномерным распеределением, то есть в каждой точке плотность равна какой-то константе, то тупа ограничивая площадь мы не поменяем это самое значение.

### Problem 7
Model and depict a sample of 500 points uniformly distributed within a given triangle without using a selection method.

### Problem 8
Model without using the selection method a sample of $500$ points uniformly distributed inside the unit circle. Picture the obtained points. Do they really fill the circle uniformly?

### Problem 9
Prove (orally) that the following algorithm (**Box-Muller algorithm**, https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform) generates a sample of independent $\mathcal{N}(0,1)$ random variables. Modify the method to exclude calls of trigonometric functions `np.sin` and `np.cos`. Using the modified method, simulate a sample size of $1000$ from a $2D$ Gaussian distribution with mean $\mu$ and covariance matrix $\Sigma$, where
$$\mu = \begin{pmatrix} 4 \\ 7 \end{pmatrix}^T, \quad \Sigma = \begin{pmatrix} 20 & -4 \\ -4 & 40 \end{pmatrix}.$$
Construct a 2D histogram of the resulting distribution. Compare the sample mean and covariance matrix with the exact values.

насчет orally: как устно не знаю, можно хар функцию посчитать - так быстрее и надежнее получится